### This file carries out orthogonal projection from a Cartesian point in space to an ellipsoid Earth.

Equatorial Radius $r_{e} =  6,378,136.6 $ $ (m)$

Polar Radius $r_{p} =   6,356,751.9 $ $ (m)$

Mathematical Expression of Earth in Cartesian coordinates:

$$ \frac{x^2}{{r_e}^2} + \frac{y^2}{{r_p}^2} + \frac{z^2}{{r_e}^2} = 1 $$

In [ ]:
import numpy as np
import utilities as ut
import json

def load_config(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)
    
config = load_config('config.json')

class Simulation():
    def __init__(self, satellite, planet, radars,
                 dt = 0.1, maxIterations = 1000000):
        
        #Satellite, Planet and Radars are objects
        
        # andre: Instantiate inside Simulation object or outside? 
        self.sat = satellite
        self.Earth = planet
        self.radars = radars

        self.dt = dt
        self.maxIter = maxIterations
        self.t = 0

        '''
        This is where the config file would be read and its contents loaded into respective variables.
        '''

    def fall(self):
        '''
        1. Declare intial conditions. -- DONE IN __init__()
        2. Enter the while loop until Crashed 
        3. Update the radar positons
        4. Update the TRUE satellite position through dynamics
        5. Radars LOS check
            5a. check for LOS for each, 
            5b. IF data update LKF update
            5c. Update Radar pings 
        6. Satellite LKF forecast  
        7. Increment simulation
        '''
    


